# A "Simple Program"


In [ ]:
#!pip install pandas_datareader # uncomment and run this to install pandas data reader

In [5]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr # you might need to install this (see above)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime

# don't copy and paste these lists and number in several places 
# (which is what the original "simple program" did... that's bad programming!)
# instead, create a var - if we change it here, it changes everywhere
stocks = ['GME','AMC','BB','BBBY','TSLA','GM','VZ']
start_yr = datetime(2021, 1, 1)

# load
stock_prices = pdr.get_data_yahoo(stocks, start=start_yr)
stock_prices = stock_prices.filter(like='Adj Close') # reduce to just columns with this in the name
stock_prices.columns = stocks # put their tickers as column names
print(stock_prices) # print

                   GME      AMC         BB       BBBY        TSLA         GM  \
Date                                                                           
2021-01-04   17.250000   2.0100   6.580000  18.030001  729.770020  40.509998   
2021-01-05   17.370001   1.9800   6.770000  19.760000  735.109985  41.660000   
2021-01-06   18.360001   2.0100   6.710000  21.030001  755.979980  42.980000   
2021-01-07   18.080000   2.0500   7.060000  18.730000  816.039978  43.320000   
2021-01-08   17.690001   2.1400   7.560000  18.940001  880.020020  43.060001   
2021-01-11   19.940001   2.2000   7.650000  20.490000  811.190002  45.009998   
2021-01-12   19.950001   2.2900   7.630000  21.530001  849.440002  47.820000   
2021-01-13   31.400000   2.1800   7.440000  23.020000  854.409973  48.730000   
2021-01-14   39.910000   2.1800   9.110000  27.340000  845.000000  51.529999   
2021-01-15   35.500000   2.3300   9.840000  25.600000  826.159973  49.970001   
2021-01-19   39.360001   3.0600  12.3500

In [2]:
# this is wide data... so if we want to create a new variable, we have to do it once for each firm...
# what if we have 1000 firms? seems tough to do...
# make long/tidy:
stock_prices = stock_prices.stack().swaplevel().sort_index().reset_index()
stock_prices.columns = ['Firm','Date','Adj Close']
stock_prices # print - now that is formatted nicely, like CRSP! 


,Firm,Date,Adj Close
0,AMC,2021-01-04,2.010000
1,AMC,2021-01-05,1.980000
2,AMC,2021-01-06,2.010000
3,AMC,2021-01-07,2.050000
4,AMC,2021-01-08,2.140000
...,...,...,...
226,VZ,2021-02-12,54.200001
227,VZ,2021-02-16,54.150002
228,VZ,2021-02-17,56.990002
229,VZ,2021-02-18,56.980000


In [ ]:
# add return var.
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby
stock_prices['ret'] = stock_prices.groupby('Firm')['Adj Close'].pct_change()
stock_prices # print - the first ret for each firm should be missing...

In [ ]:
(stock_prices
     .assign(ret=1+stock_prices.ret)              # gross returns
     .set_index(['Date','Firm'])['ret'].unstack() # convert to wide format
     .cumprod()
     .plot()
)